# Library

In [ ]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm==0.5.4 einops joblib icecream  -qq -U

     |████████████████████████████████| 21.8 MB 1.3 MB/s 
     |████████████████████████████████| 102 kB 72.6 MB/s 
     |████████████████████████████████| 431 kB 73.9 MB/s 
     |████████████████████████████████| 180 kB 75.2 MB/s 
     |████████████████████████████████| 143 kB 73.3 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 


In [ ]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from sklearn.metrics import f1_score

# ENV

In [ ]:

# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
# FP16 = True
FP16 = False


CONFIG = dict(
    seed=42,
    nickname='deit finecutmix',
    backbone='deit_base_distilled_patch16_384',
    embedder=None,
    train_batch_size=8,
    valid_batch_size=16,
    img_size=384,
    num_epochs=20,
    early_stopping=False,
    early_stopping_step=5,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes=25,
    weight_decay=1e-6,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition='lg',
    _wandb_kernel='deb'
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# SET SEED 

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(CONFIG['seed'])


# Read the Data


In [ ]:
import wandb
run = wandb.init(project="lg",
                 entity="jiwon7258",
                 config=CONFIG,
                 job_type='train',
                #  id='3199pcx4',
                #  resume='must',
                 )
dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg_train:v0', type='dataset')

run.name = CONFIG['nickname']

# Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact lg_train:v0, 918.24MB. 17301 files... Done. 0:0:0


In [ ]:
TRAIN_PATH = dataset_dir
# TEST_PATH = dataset_dir / 'test'

# Augmentations

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1,
                           rotate_limit=90, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15,
                   b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()], p=1.),

    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(),
        ToTensorV2()], p=1.)
}


# Dataset

In [ ]:
train_csv = sorted(glob(str(TRAIN_PATH / '*/*.csv')))
train_jpg = sorted(glob(str(TRAIN_PATH / '*/*.jpg')))
train_json = sorted(glob(str(TRAIN_PATH / '*/*.json')))


crops = []
diseases = []
risks = []
labels = []

for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
    
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
        
label_unique = sorted(np.unique(labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in labels] # len = train_len

In [ ]:
train_jpg = np.array(train_jpg)
train_labels = np.array(train_labels)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, train_img, train_label, transforms=None):
        self.imgs = train_img
        self.labels = train_label
        self.transforms = transforms
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index):
        img_path = self.imgs[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return img, target
    
# trainDataset = CustomDataset(X_train, y_train, transforms = data_transforms['train'])
# trainDataloader = DataLoader(
#     trainDataset, batch_size=CONFIG['train_batch_size'], shuffle=True)

# validDataset = CustomDataset(X_val, y_val, transforms = data_transforms['valid'])
# validDataloader = DataLoader(validDataset, batch_size = CONFIG['valid_batch_size'], shuffle = True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
train_datasets = []
valid_datasets = []
train_dataloaders = []
valid_dataloaders = []

skf = StratifiedKFold(n_splits = 5)

for step, (train_index, val_index) in enumerate(skf.split(X = train_jpg, y= train_labels)):
    X_train = train_jpg[train_index]
    y_train = train_labels[train_index]
    X_val = train_jpg[val_index]
    y_val = train_labels[val_index]
    train_datasets.append(CustomDataset(
        X_train, y_train, transforms=data_transforms['train']))
    valid_datasets.append(CustomDataset(
        X_val, y_val, transforms=data_transforms['valid']))
    train_dataloaders.append(DataLoader(
        train_datasets[step], batch_size=CONFIG['train_batch_size'], shuffle=True)
    )
    valid_dataloaders.append(
        DataLoader(
            valid_datasets[step], batch_size=CONFIG['valid_batch_size'], shuffle=True)
    )


# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 768
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        if isinstance(features, tuple):
          features = features[0]
        output = self.fc(features)
        return output


model = Model(CONFIG['backbone'], CONFIG['embedder'], pretrained = True)
model.to(CONFIG['device'])
;

''

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])


In [ ]:
def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss()(logits.view(-1,CONFIG['num_classes']), targets.view(-1))

# CutMix

In [ ]:
def cutmix(img, target):
    """ 
    img : (bs, C, H, W)
    target
        - (bs,)
        - integer scalar
    """
    batch_size, C, H, W, = img.shape
    # ic(img.shape)

    img_a = img
    target_a = target
    img_b = img
    target_b = target

    mask = np.arange(batch_size)
    mask = np.random.permutation(mask)
    # ic(mask)
    img_b = img_a[mask]
    target_b = target_a[mask]
    # ic(target, target_b)

    lam = np.random.uniform(low=0.3, high=0.7)
    r_x = np.random.uniform(low=0, high=W)
    r_y = np.random.uniform(low=0, high=H)
    r_w = W * np.sqrt(1 - lam)
    r_h = H * np.sqrt(1 - lam)
    ic(lam, r_x, r_y, r_w, r_h)
    x1 = np.int(np.clip((r_x - r_w) / 2, 0, W))
    x2 = np.int(np.clip((r_x + r_w) / 2, 0, W))
    y1 = np.int(np.clip((r_y - r_h) / 2, 0, H))
    y2 = np.int(np.clip((r_y + r_h) / 2, 0, H))
    ic(x1, x2, y1, y2)

    img_a[:, :, y1:y2, x1:x2] = img_b[:, :, y1:y2, x1:x2]

    # Adjust lambda to exact ratio

    lam = 1 - (x2 - x1) * (y2 - y1) / float(W * H)

    return img_a, target_b, lam


   # Training Function

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    if(FP16):
        scaler = amp.GradScaler()

    losses = AverageMeter()
    accuracy = AverageMeter()
    f1 = AverageMeter()

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img, target_b, lam = cutmix(img, target)

        img = img.to(device)
        target = target.to(device)
        target_b = target_b.to(device)

        batch_size = img.shape[0]

        if(FP16):
            with amp.autocast(enabled=True):
                logits = model(img)
                loss = criterion(logits, target) * lam + \
                    criterion(logits, target_b) * (1-lam)

                # loss를 Scale
                # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
                scaler.scale(loss).backward()
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
                # otherwise, optimizer.step() is skipped.
                scaler.step(optimizer)

                # Updates the scale for next iteration.
                scaler.update()

        else:
            logits = model(img)
            loss = criterion(logits, target) * lam + \
                criterion(logits, target_b) * (1-lam)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        losses.update(loss.item())

        # logits
        logits = logits.detach().cpu()

        # acc, f1
        probs = torch.softmax(logits, dim = -1)
        output = np.argmax(probs, axis=-1)
        output_b = np.argsort(probs)[:,-2]
        if (lam >= 0.5):
            step_acc = np.mean(
                output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
            step_f1 = f1_score(output.view(-1).numpy(),
                               target.view(-1).detach().cpu().numpy(), average='macro')
            step_acc_b = np.mean(
                output_b.view(-1).numpy() == target_b.view(-1).detach().cpu().numpy())
            step_f1_b = f1_score(output_b.view(-1).numpy(),
                                 target_b.view(-1).detach().cpu().numpy(), average='macro')
        else:
            step_acc = np.mean(
                output.view(-1).numpy() == target_b.view(-1).detach().cpu().numpy())
            step_f1 = f1_score(output.view(-1).numpy(),
                               target_b.view(-1).detach().cpu().numpy(), average='macro')
            step_acc_b = np.mean(
                output_b.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
            step_f1_b = f1_score(output_b.view(-1).numpy(),
                                 target.view(-1).detach().cpu().numpy(), average='macro')

                                 
        step_acc = step_acc * lam + step_acc_b * (1-lam)
        step_f1 = step_f1 * lam + step_f1_b * (1-lam)

        accuracy.update(step_acc)
        f1.update(step_f1)

        # loss
        train_loss = losses.avg
        train_acc = accuracy.avg
        train_f1 = f1.avg

        bar.set_postfix(
            Epoch=epoch, Train_Loss=train_loss, LR=optimizer.param_groups[
                0]["lr"], accuracy=train_acc, f1=train_f1
        )

    # Garbage Collector
    gc.collect()

    return losses.avg, accuracy.avg, f1.avg


   # Validation Function

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    losses = AverageMeter()
    accuracy = AverageMeter()
    f1 = AverageMeter()

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (img, target) in bar:
        img = img.to(device)
        target = target.to(device)

        batch_size = img.shape[0]

        logits = model(img)
        loss = criterion(logits, target)

        # loss.item()은 loss를 Python Float으로 반환
        losses.update(loss.item())

        # logits
        logits = logits.detach().cpu()

        output = np.argmax(torch.softmax(logits, dim=-1), axis=-1)
        step_acc = np.mean(
            output.view(-1).numpy() == target.view(-1).detach().cpu().numpy())
        step_f1 = f1_score(output.view(-1).numpy(),
                           target.view(-1).detach().cpu().numpy(), average='macro')

        accuracy.update(step_acc)
        f1.update(step_f1)

        # loss
        val_loss = losses.avg
        val_acc = accuracy.avg
        val_f1 = f1.avg

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[
                0]["lr"], accuracy=val_acc, f1=val_f1
        )

    gc.collect()

    return losses.avg, accuracy.avg, f1.avg


In [ ]:

def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
    START_EPOCH=0,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(START_EPOCH, START_EPOCH + num_epochs):
        gc.collect()

        fold_num = 5
        fold = epoch % fold_num

        # for fold in range(fold_num) :

        trainDataloader = train_dataloaders[fold]
        validDataloader = valid_dataloaders[fold]

        train_train_loss, train_accuracy, train_f1 = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=trainDataloader,
            device=device,
            epoch=epoch,
        )

        val_loss, val_accuracy, val_f1 = valid_one_epoch(
            model, validDataloader, device=device, epoch=epoch
        )

        history[f"{metric_prefix}Train Loss"].append(train_train_loss)
        history[f"{metric_prefix}Train Accuracy"].append(train_accuracy)
        history[f"{metric_prefix}Train F1"].append(train_f1)
        history[f"{metric_prefix}Valid Loss"].append(val_loss)
        history[f"{metric_prefix}Valid Accuracy"].append(val_accuracy)
        history[f"{metric_prefix}Valid F1"].append(val_f1)

        # Log the metrics
        wandb.log(
            {
                f"{metric_prefix}Train Loss": train_train_loss,
                f"{metric_prefix}Valid Loss": val_loss,
                f"{metric_prefix}Train Accuracy": train_accuracy,
                f"{metric_prefix}Valid Accuracy": val_accuracy,
                f"{metric_prefix}Train F1": train_f1,
                f"{metric_prefix}Valid F1": val_f1,
            }
        )

        print(f"Valid Loss : {val_loss}")

        torch.save(model.state_dict(), f'{CONFIG["nickname"]}last.bin')
        wandb.save(f'{CONFIG["nickname"]}last.bin')

        # deep copy the model
        if val_loss <= best_loss:
            early_stop_counter = 0

            print(
                f"Validation Loss improved( {best_loss} ---> {val_loss}  )"
            )

            # Update Best Loss
            best_loss = val_loss

            # Update Best Model Weight
            # run.summary['Best RMSE'] = best_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            # PATH = "{}epoch{:.0f}_Loss{:.4f}.bin".format(
            #     file_prefix, epoch, best_loss)
            # torch.save(model.state_dict(), PATH)
            # # Save a model file from the current directory
            # wandb.save(PATH)

            print(f"Model Saved")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

        START_EPOCH = epoch + 1
        # break

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    return model, history


In [ ]:
wandb.restore('epoch19_Loss0.0245.bin', 'jiwon7258/lg/5ikpl9uv', root='./')
model.load_state_dict(torch.load('epoch19_Loss0.0245.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

# Fast Start : 5 epoch

In [ ]:
for param in model.backbone.named_parameters():
    param[1].requires_grad = False

In [ ]:
ic.disable()
run_training(
    model=model,
    optimizer=optimizer,
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    device=device,
    num_epochs=5,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
    START_EPOCH=0,
);


[INFO] Using GPU:Tesla P100-PCIE-16GB



100%|██████████| 73/73 [00:34<00:00,  2.09it/s, Epoch=0, LR=3.93e-6, Valid_Loss=0.00742, accuracy=0.998, f1=0.996]


Valid Loss : 0.007423872112979066
Validation Loss improved( inf ---> 0.007423872112979066  )
Model Saved


100%|██████████| 73/73 [00:34<00:00,  2.13it/s, Epoch=1, LR=8.86e-5, Valid_Loss=0.0182, accuracy=0.999, f1=0.998]


Valid Loss : 0.018220911982584083


...

Valid Loss : 0.04974257264745562


100%|██████████| 73/73 [00:34<00:00,  2.14it/s, Epoch=3, LR=5.98e-5, Valid_Loss=0.0646, accuracy=0.999, f1=0.998]


Valid Loss : 0.06462013287699386


100%|██████████| 73/73 [00:34<00:00,  2.13it/s, Epoch=4, LR=5.82e-5, Valid_Loss=0.0812, accuracy=0.999, f1=0.997]


Valid Loss : 0.08117050886766551
Training complete in 0h 15m 35s
Best Loss: 0.0074


# 400 epochs

In [ ]:
optimizer = torch.optim.Adam(
    params=model.parameters(), lr=5e-6, weight_decay=CONFIG['weight_decay'])

In [ ]:
for param in model.backbone.named_parameters():
    param[1].requires_grad = True


In [40]:
ic.disable()
run_training(
    model=model,
    optimizer=optimizer,
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer, T_max=CONFIG['T_max'], eta_min=CONFIG['min_lr']),
    # scheduler=None,
    device=device,
    num_epochs=400,
    metric_prefix="",
    file_prefix="",
    early_stopping=CONFIG['early_stopping'],
    early_stopping_step=CONFIG['early_stopping_step'],
    START_EPOCH=20,
)


[INFO] Using GPU:Tesla P100-PCIE-16GB



100%|██████████| 73/73 [00:33<00:00,  2.16it/s, Epoch=20, LR=1.12e-6, Valid_Loss=0.023, accuracy=0.999, f1=0.998]


Valid Loss : 0.022973638716829965
Validation Loss improved( inf ---> 0.022973638716829965  )
Model Saved


100%|██████████| 73/73 [00:33<00:00,  2.16it/s, Epoch=21, LR=4.54e-6, Valid_Loss=0.0235, accuracy=0.999, f1=0.998]


Valid Loss : 0.023529678200409836


...

Valid Loss : 0.0009340649340716383
Validation Loss improved( 0.0009561500544755477 ---> 0.0009340649340716383  )
Model Saved


100%|██████████| 73/73 [00:34<00:00,  2.10it/s, Epoch=201, LR=5e-6, Valid_Loss=0.000999, accuracy=1, f1=1]


Valid Loss : 0.0009990072420842894


 18%|█▊        | 53/289 [01:05<04:50,  1.23s/it, Epoch=202, LR=2.94e-6, Train_Loss=0.524, accuracy=0.911, f1=0.888]


KeyboardInterrupt: ignored

In [41]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe84b30ead0>> (for pre_run_cell):


Exception: ignored

Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe84b30ead0>> (for post_run_cell):


Exception: ignored

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_run.py", line 166, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 114, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 378, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 213, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/wa